In [ ]:
partition = 100

In [ ]:
import sys
from itertools import product

In [ ]:
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))
from train import main

In [ ]:
experiment_dir = os.getcwd()
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "../..")))

In [ ]:
n_tree_values = [100]
tree_depth_values = [4]
hidden_dim = [1024, 768]
batch_size_values = [256, 512]
tree_feature_rates = [0.1, 0.2]
feat_dropouts = [0.0, 0.1, 0.2]
lrs = [0.01]


In [ ]:
best_score = 0
best_config = {}

for n_tree, t_depth, hd, batch_size, tfr, do, lr in product(n_tree_values, tree_depth_values, hidden_dim, batch_size_values, tree_feature_rates, feat_dropouts, lrs):
    print(f"Running: n_tree={n_tree}, t_depth={t_depth}, hd={hd}, batch_size={batch_size} tree_feature_rates={tfr}, feat_dropouts={do}, hd={hd}, lrs={lr}")
    results_dir = os.path.join(experiment_dir, "results")
    os.makedirs(results_dir, exist_ok=True)

    sys.argv = [
        'train.py',
        '-dataset', f'gtd{partition}',
        '-n_class', '30',
        '-gpuid', '0',
        '-n_tree', str(n_tree),
        '-tree_depth', str(t_depth),
        '-batch_size', str(batch_size),
        '-hidden_dim', str(hd),
        '-epochs', '300',
        '-verbose', '0',
        '-tree_feature_rate', str(tfr),
        '-feat_dropout', str(do),
        '-lr', str(lr),
        '-jointly_training',
        '-searching', '1',
        '-results_dir', results_dir
    ]

    complete = main()
    print(complete)

    
    result_file = os.path.join(results_dir, f"result_gtd{partition}.txt")
    try:
        with open(result_file, "r") as f:
            lines = f.readlines()
            for line in lines:
                if "Best Accuracy" in line:
                    acc = float(line.split()[2])
                    if acc > best_score:
                        best_score = acc
                        best_config = {
                            'n_tree': n_tree,
                            'tree_depth': t_depth,
                            'batch_size': batch_size,
                            'hidden_dim': hd,
                            'tree_feature_rate': tfr,
                            'feat_dropout': do,
                            'lr': lr
                        }
    except FileNotFoundError:
        print(f"ERROR: Result file not found: {result_file}")
print("Best hyperparameter configuration:")
print(best_config)
print(f"Best accuracy: {best_score}")

try:
    with open(result_file, "a") as f:
        for k, v in best_config.items():
            f.write(f"{k}: {v}, ")
except Exception as e:
    print(f"ERROR: Failed to write best_config to result file: {e}")
